In [ ]:
# Importing the numpy library
import numpy as np

In [ ]:
# printoptions helps us improve the way we display the output on the screen 

np.set_printoptions(suppress = True, linewidth = 100, precision = 2)

In [ ]:
# Importing the dataset using np.genfromtxt function (to avoid errors if there is missing values) & display it

raw_data_np = np.genfromtxt("loan-data.csv", 
                            delimiter = ";", 
                            skip_header = 1, 
                            autostrip = True)
raw_data_np

## 1. Checking for Incomplete Data

In [ ]:
# See how many missing values we have (np.isnan() provides an array of True (it is a missing value) or False, 
#with the sum() function, the number of 'True' will be summed

np.isnan(raw_data_np).sum()

In [ ]:
# Create a variable to temporary fill the missing entries of the dataset (temporary_fill)
# It will later be raplaced with the temporary_mean variable (holds the means for every column)

temporary_fill = np.nanmax(raw_data_np).round(2) + 1
temporary_mean = np.nanmean(raw_data_np, axis = 0)

In [ ]:
#Store the max and the min in an array,  that will be useful later on

temporary_stats = np.array([np.nanmin(raw_data_np, axis = 0), 
                            temporary_mean, 
                            np.nanmax(raw_data_np, axis = 0)])

Numpy does not like us storing diffferent data types in the same array since it limits what we can do with the dataset. We will thus split the data set into two datasets: one containing the numerical values, and one containing the string values

## 2. Splitting the Dataset & Reimporting it

In [ ]:
# Get the index of the columns with strings values (= mean is nan)
# np.argwhere() will return the index of the column in the original dataset
column_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()
column_strings

In [ ]:
#Get the index of the columns with  numeric values
column_numeric = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
column_numeric

In [ ]:
#Importing the string data set 
loan_data_strings = np.genfromtxt("loan-data.csv",
                                  delimiter = ";",
                                  skip_header = 1,
                                  autostrip = True,
                                  usecols = column_strings,
                                  dtype = str)
loan_data_strings

In [ ]:
#Importing the numeric data set and defining the missing values as temporary_fill
loan_data_numeric = np.genfromtxt("loan-data.csv",
                                  delimiter = ";",
                                  skip_header = 1,
                                  autostrip = True,
                                  usecols = column_numeric,
                                  filling_values = temporary_fill)
loan_data_numeric


In [ ]:
#Store the header information
#skip_footer = raw_data_np.shape[0] : will not import all rows after the header
header_full = np.genfromtxt("loan-data.csv",
                            delimiter = ";",
                            skip_footer = raw_data_np.shape[0],
                            autostrip = True,
                            dtype = str)
header_full

In [ ]:
#Store the header of each data type in a seperate variable using their indexes
header_strings, header_numeric = header_full[column_strings], header_full[column_numeric]

In [ ]:
header_strings

In [ ]:
header_numeric 

## 3. Creating Checkpoints:

In [ ]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)

In [ ]:
checkpoint_test = checkpoint('checkpoint-test', 
                             header_strings, 
                             loan_data_strings)

## 4. Manipulating String Columns

In [ ]:
header_strings

### 4.1 Issue Date

In [ ]:
#Renaming the first header to a more explicitname 
header_strings[0] = "issue_date"

In [ ]:
#Visualise the data in the issue date column (Format: Month - Year)
loan_data_strings[:,0]

In [ ]:
#All loans are from the year -15 --> Excess data
np.unique(loan_data_strings[:,0])

In [ ]:
#Stripping the "-15" from the column and overwritting the given column 
loan_data_strings[:,0] = np.chararray.strip(loan_data_strings[:,0], "-15")

In [ ]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [ ]:
#Stocking the months as numerical values (less memory space used & order) using 
#a loop and the np.where() function 
for i in range(13):
        loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                          i,
                                          loan_data_strings[:,0])

In [ ]:
#We transformed this string column into a numerical one.
np.unique(loan_data_strings[:,0])

### 4.2 Loan Status

In [ ]:
header_strings[1] = "loan_status"

In [ ]:
loan_data_strings[:,1]

In [ ]:
np.unique(loan_data_strings[:,1])

In [ ]:
np.unique(loan_data_strings[:,1]).size

1. Good loan status (not defaulted): "Fully paid", "Current", "Issued", "In grace period", "Late (16-30 days)" = 1 

2. Bad loan status (defaulted): "Charged off", "Default", "Late (31-120 days)" ,missing values = 0

In [ ]:
#Store the loan status considered as bad in a separate array
loan_status_bad = np.array(['Charged Off', 'Default', 'Late(31-120 days)'])

In [ ]:
#np.isin(): checks if the elements in  interval 2 are in interval 1
#np.where(): replace the bad loan status with the number 0, otherwise (good loan status) 1.
loan_data_strings[:,1]= np.where(np.isin(loan_data_strings[:,1], 
                                         loan_status_bad),
                                 0,
                                 1)

In [ ]:
#We created a dummy variable, which is what was asked from us.
np.unique(loan_data_strings[:,1])

In [ ]:
loan_data_strings[:,1]

### 4.3 Term

In [ ]:
header_strings[2] = "term_months"

In [ ]:
loan_data_strings[:,2]

In [ ]:
np.unique(loan_data_strings[:,2])

In [ ]:
loan_data_strings[:,2]= np.chararray.strip(loan_data_strings[:,2], " months")

In [ ]:
#We assume the worst: Missing values = 60 month (long period in itself for a loan
#to pay off, worst term we can potentially end up with)
loan_data_strings[:,2] = np.where(loan_data_strings[:,2] == '', 
                                  '60',
                                  loan_data_strings[:,2])

In [ ]:
np.unique(loan_data_strings[:,2])
#As expected, we get only 36 and 60 
#when we only have two possible numerical outcomes for a given column,
#this immediately rings a bell that we could just use one in zero instead.
#always ask the question

### 4.4 Grade and Subgrade

In [ ]:
loan_data_strings[:,3]

In [ ]:
np.unique(loan_data_strings[:,3])

In [ ]:
loan_data_strings[:,4]

In [ ]:
#For every element in grade, there are 5 subelements in subgrade
#The information that we obtain in the grade column can also be obtained in the 
#subgrade column.
np.unique(loan_data_strings[:,4])

#### 4.4.1 Filling Sub Grade & Deleting Grade

In [ ]:
#We assign the lowest, worst subgrade (5) to the missing values in the third column
#when we have a grade for this value.
for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),
                                      i + '5',
                                      loan_data_strings[:,4])

In [ ]:
#For the missing values for both grade and subgrade, we assume the worst
#We will create a lower value than G5 
loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == ''),
                                      "H1",
                                      loan_data_strings[:,4])

In [ ]:
#We cleaned the Subgrade column, we don't have any missing values anymore (H1 instead)
np.unique(loan_data_strings[:,4]) 

In [ ]:
#We remove the grade (3) column (axis = 1)
#since we have this information in the subgrade column
loan_data_strings = np.delete(loan_data_strings,3, axis = 1)

In [ ]:
#Delete this header of this column as well
header_strings = np.delete(header_strings, 3)

#### 4.4.2 Converting Sub Grade into a Numerical Column

In [ ]:
np.unique(loan_data_strings[:,3])

In [ ]:
#Constitute a dictionary with keys equal to the subgrade (string) and values equal 
#to a rank of trustworthiness (A1 = 1) 
keys = list(np.unique(loan_data_strings[:,3]))                         
values = list(range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1)) 
dict_sub_grade = dict(zip(keys, values))

In [ ]:
dict_sub_grade

In [ ]:
list(dict_sub_grade.keys())[0]

In [ ]:
list(dict_sub_grade.values())[0]

In [ ]:
#Iterate for each subgrade and substitute each one with its associated 
#numerical value (Key)
for i in np.unique(loan_data_strings[:,3]):
        loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i, 
                                          dict_sub_grade[i],
                                          loan_data_strings[:,3])

In [ ]:
##We transformed this string column into a numerical one.
np.unique(loan_data_strings[:,3])

### 4.5 Verification Status

In [ ]:
header_strings

In [ ]:
loan_data_strings[:,4]

In [ ]:
np.unique(loan_data_strings[:,4])


1. Good verification status (loan applications that include investor backing)= 1
2. Bad verification status = "Not verified", '' = 0

In [ ]:
verification_status_bad = ['Not verified', '']

In [ ]:
loan_data_strings[:,4] = np.where(np.isin(loan_data_strings[:,4], verification_status_bad), 
                                  0, 
                                  1)

In [ ]:
#We transformed this string column into a numerical one.
np.unique(loan_data_strings[:,4])

### 4.6 URL

In [ ]:
loan_data_strings[:,5]

In [ ]:
#Keep only the unique elements in the url (loan_id number)
loan_data_strings[:,5]= np.chararray.strip(loan_data_strings[:,5], 
                                           "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

In [ ]:
#Load_ID (string) = ID (float)
loan_data_strings[:,5]

In [ ]:
loan_data_numeric[:,0]

In [ ]:
loan_data_numeric[:,0].astype(dtype= np.int32)

In [ ]:
loan_data_strings[:,5].astype(dtype = np.int32)

In [ ]:
#Both arrays are identical
np.array_equal(loan_data_numeric[:,0].astype(dtype= np.int32),loan_data_strings[:,5].astype(dtype = np.int32))

In [ ]:
#We can get rid of the URL column since it odoes not hold any additional value.
loan_data_strings = np.delete(loan_data_strings, 5, axis = 1)
header_strings = np.delete(header_strings, 5)

### 4.7 State Address

In [ ]:
header_strings[5] = "state_addresss"

In [ ]:
loan_data_strings[:,5]

In [ ]:
np.unique(loan_data_strings[:,5], return_counts = True)

In [ ]:
np.unique(loan_data_strings[:,5]).size

In [ ]:
np.unique(loan_data_strings[:,5], return_counts = True)

In [ ]:
#We sort the state name and count in ascending order
states_names, states_count = np.unique(loan_data_strings[:,5], return_counts = True)
states_count_sorted = np.argsort(-states_count)
states_names[states_count_sorted], states_count[states_count_sorted]

In [ ]:
loan_data_strings[:,5] = np.where(loan_data_strings[:,5] =='', 0, loan_data_strings[:,5])

In [ ]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

In [ ]:
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_west), 1, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_south), 2, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_midwest), 3, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_east), 4, loan_data_strings[:,5])

In [ ]:
np.unique(loan_data_strings[:,5])

## 5. Converting String Column to Numbers

In [ ]:
loan_data_strings

As of now, we converted each string data we have into numeric values stored as text, we now have to convert them to a numeric data type

In [ ]:
loan_data_strings = loan_data_strings.astype(np.int64)

In [ ]:
loan_data_strings

### 5.1 Checkpoint 1: Strings

In [ ]:
checkpoint_strings= checkpoint("Checkpoint- Strings", header_strings, loan_data_strings)

In [ ]:
checkpoint_strings["header"]

In [ ]:
checkpoint_strings["data"]

In [ ]:
np.array_equal(checkpoint_strings["data"], loan_data_strings)

## 6. Manipulating Numeric Columns

In [ ]:
loan_data_numeric

In [ ]:
header_numeric

In [ ]:
#Check for missing values = check if any of the elements in the column is equal to temporary_fill
temporary_fill

In [ ]:
#We must substitute the missing values with the worst possible values
np.isnan(loan_data_numeric).sum()

In [ ]:
#We will use the statistics that we stored in temporary_stats (min, max, mean)
#We don't have any statstics in the string columns (string values)
temporary_stats[:, column_numeric]

### 6.1 ID

In [ ]:
loan_data_numeric[:,0]

In [ ]:
np.isin(loan_data_numeric[:,0], temporary_fill)

In [ ]:
#No missing values (normal since it is the ID column)
np.isin(loan_data_numeric[:,0], temporary_fill).sum()

### 6.2 Funded Amount

In [ ]:
loan_data_numeric[:,2]

In [ ]:
#Worst case scenario = Minimum value (first row of the temporary_stats)
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill, 
                                  temporary_stats[0, column_numeric[2]],
                                  loan_data_numeric[:,2])
loan_data_numeric[:,2]

### 6.3 Loaned Amount, Interest Rate, Total Payment, Installment

In [ ]:
header_numeric

In [ ]:
#We replace the temporary_fill with the maximum value (third row of the temporary_stats)
for i in [1,3,4,5]: 
    loan_data_numeric[:,i] = np.where(loan_data_numeric[:,i] == temporary_fill, 
                                  temporary_stats[2, column_numeric[i]],
                                  loan_data_numeric[:,i])

In [ ]:
loan_data_numeric

### 6.4 Currency Change

#### 6.4.1 The Exchange Rate

We filled out all missing values properly, we will now convert all the Dollar signes to Euros. To do this, we need the exchange rate between the 2 currencies at the time of the loan application

EUR-USD.csv contains the average monthly exchange rates for 2015

In [ ]:
#In our case, we only care about the close daily exchange rate (adjusted closing prices), third column
EUR_USD= np.genfromtxt("EUR-USD.csv",
                       delimiter= ",", 
                       autostrip = True, 
                       skip_header = 1, 
                       usecols = 3)
EUR_USD

In [ ]:
header_strings[0]

In [ ]:
#Months in which the loan was issued (0= missing values)
loan_data_strings[:,0]

In [ ]:
#Substitute the month value with the exchange rate at that time 
exchange_rate = loan_data_strings[:,0]
for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)    

In [ ]:
#Substitute the missing values (= 0) with the mean of the array
exchange_rate = np.where(exchange_rate == 0,
                             np.mean(EUR_USD),
                             exchange_rate)   

In [ ]:
exchange_rate

In [ ]:
exchange_rate.shape

In [ ]:
loan_data_numeric.shape

In [ ]:
#We need to reshape the exchange rate array, transform it into an array
exchange_rate = np.reshape(exchange_rate, (10000,1))

In [ ]:
#We stack the exchange_rate array to the loan_data_numeric array = 
#we add a column to the dataset
loan_data_numeric = np.hstack((loan_data_numeric, exchange_rate))

In [ ]:
header_numeric  = np.concatenate((header_numeric, np.array(['exchange_rate'])))

#### 6.4.2 From USD to EUR

In [ ]:
header_numeric

In [ ]:
columns_dollar = np.array([1,2,4,5])

In [ ]:
loan_data_numeric[:, [columns_dollar]]

In [ ]:
#We divide each column with a dollar value by the exchange rate to obtain the
#value in eur , reshape it to obtain an array & stack it to the data set. 
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, 
                                   np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

In [ ]:
#We have five new columns in the array.
loan_data_numeric.shape

#### 6.4.3 Expanding the header with the five new columns created

In [ ]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])

In [ ]:
header_additional

In [ ]:
header_numeric = np.concatenate((header_numeric, header_additional))

In [ ]:
header_numeric[columns_dollar] = np.array([column_name + '_USD' for column_name in header_numeric[columns_dollar]])

In [ ]:
header_numeric

In [ ]:
#Each EUR column will follow its corresponding US column
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [ ]:
header_numeric = header_numeric[columns_index_order]

In [ ]:
loan_data_numeric = loan_data_numeric[:,columns_index_order]

In [ ]:
loan_data_numeric 

For now, we have : 
1. Appropriately filled out any missing values
2. Added exchange rates for each applicant (account)
3. Created EUR versions of the 5 monetary values

### 6.5 Interest Rate

In [ ]:
header_numeric

In [ ]:
loan_data_numeric[:,5]

In [ ]:
#Usually, it's better to have the interest rates between 0 and 1, we will thus
#divide it by 100
loan_data_numeric[:,5] = loan_data_numeric[:,5]/100

## 7. Checkpoint 2: Numeric

In [ ]:
checkpoint_numeric = checkpoint("Checkpoint-Numeric", header_numeric, loan_data_numeric)

In [ ]:
checkpoint_numeric['header'], checkpoint_numeric['data']

## 8. Creating the "Complete" Dataset & Sorting it

In [ ]:
checkpoint_strings['data'].shape

In [ ]:
checkpoint_numeric['data'].shape

In [ ]:
np.hstack((checkpoint_numeric['data'], checkpoint_strings['data'])).shape

In [ ]:
loan_data = np.hstack((checkpoint_numeric['data'], checkpoint_strings['data']))

In [ ]:
np.isnan(loan_data).sum()

In [ ]:
header_full = np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))

We want to rearrange the entire dataset according to the values in the first column (ID)

In [ ]:
np.argsort(loan_data[:,0])

In [ ]:
loan_data = loan_data[np.argsort(loan_data[:,0])]

In [ ]:
loan_data 

In [ ]:
np.argsort(loan_data[:,0])

## 9. Storing the New Dataset

In [ ]:
loan_data = np.vstack((header_full, loan_data))

In [ ]:
np.savetxt("loan-data-preprocessed.csv", 
           loan_data, 
           fmt = "%s", 
           delimiter = ",")